In [1]:
from sklearn.datasets import fetch_20newsgroups

categories = [
    "alt.atheism",
    "rec.autos",
    "comp.graphics",
    "sci.space",
]

data_train = fetch_20newsgroups(
    subset="train",
    categories=categories,
    shuffle=True,
    random_state=42,
    remove=(),
)
data_test = fetch_20newsgroups(
    subset="test",
    categories=categories,
    shuffle=True,
    random_state=42,
    remove=(),
)

import torch
from sklearn.feature_extraction.text import TfidfVectorizer
!pip install bertviz
from bertviz import head_view, model_view

# order of labels in `target_names` can be different from `categories`
target_names = data_train.target_names

# split target in a training set and a test set
y_train, y_test = data_train.target, data_test.target

# Extracting features from the training data using a sparse vectorizer
vectorizer = TfidfVectorizer(
    sublinear_tf=True, max_df=0.5, min_df=5, stop_words="english"
)
X_train = vectorizer.fit_transform(data_train.data)

# Extracting features from the test data using the same vectorizer
X_test = vectorizer.transform(data_test.data)

feature_names = vectorizer.get_feature_names_out()

seq = X_test[:32,:]
b = seq.toarray()
att = b@b.T
att = att.reshape(1,1,32,32)
att = torch.tensor(att)
counts = [0,0,0,0]
tokens = []
for i in y_test[:32]:
  token = target_names[i] + ' ' + str(counts[i])
  tokens.append(token)
  counts[i]+=1
head_view((att,), tokens)


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


<IPython.core.display.Javascript object>

### A) Essa visualização representa uma matriz de atenção 32x32 aplicada a um conjunto de 32 amostras compostas por uma frase (vetor) e um output (classe). Essa matriz de atenção é construída a partir do produto entre b e bT, onde b é uma matriz com 32 linhas (cada amostra) e 8025 colunas (tamanho do vocabulário de treino definido pelo modelo de vetorização). Note que essa matriz att (definida pelo produto interno 2 a 2 entre os vetores que representam o input) representa a relação entre as amostras e portanto carrega consigo a relação entre o output. Assim, plotamos como cada output relaciona com os outros de acordo com a matriz de atenção. Podemos abstrair essa atenção como uma representação de correlação entre outputs.


### OBS: Note que o input aqui é uma matriz de 8025 colunas representando uma frase/texto que é basicamente uma contagem da quantidade de vezes que uma palavra i aparece na frase ponderado pelo inverso das vezes que essa palavra aparece no conjunto completo dos dados

### B) Note que o cosseno captura uma noção de distância compreensível quando é possível se obter intersecção entre os vetores na direção definida por seus versores. Do contrário, quando tais direções definem subespaços reversos, o cosseno se torna de difícil interpretação. Além disso, quando estrapolamos o angulo para valores fora do primeiro quadrante, notamos que o cosseno passa a retornar valores negativos, distorcendo a interpretabilidade da métrica e retornando "distancias" de mesmo modulo apesar dos vetores estarem mais distantes.

### Ao limitar-se o dominio do problema (ou caso distancias negativas sejam uma metrica desejavel), o cosseno pode se mostrar mais vantajoso. Do contrario, a distancia euclidiana se adequa melhor.

In [2]:
# c)

X_train = torch.tensor(X_train.toarray(), dtype=torch.float32).cuda()
X_train = (X_train-X_train.mean())/X_train.std()
y_train = torch.tensor(y_train, dtype=torch.float32).cuda()
X_test = torch.tensor(X_test.toarray(), dtype=torch.float32).cuda()
X_test = (X_test-X_test.mean())/X_test.std()
y_test = torch.tensor(y_test, dtype=torch.float32).cuda()

torch.manual_seed(0)

n_epochs = 5
lr = 1e-5
sequence_length = 32

def init_scale(fan_in):
    return (2/fan_in)**.5

D = X_train.shape[1]

A_K = torch.randn((D,D)).cuda()#*init_scale(D)
A_Q = torch.randn((D,D)).cuda()#*init_scale(D)

A_K.requires_grad = True
A_Q.requires_grad = True

optimizer = torch.optim.Adam([A_K, A_Q], lr = lr)
softmax = torch.nn.Softmax(dim=0)
cos = torch.nn.CosineSimilarity(dim=0)
losses = []
for epoch in range(n_epochs):
  permutation = torch.randperm(X_train.shape[0])
  for i in range(0,X_train.shape[0], sequence_length):
    optimizer.zero_grad()
    indices = permutation[i:i+sequence_length]
    if len(indices)!=sequence_length:
      continue
    seq_X, seq_Y = X_train[indices], y_train[indices]

    K = A_K@(seq_X.T)
    Q = A_Q@(seq_X.T)
    att = softmax((K.T@Q)/(D**(1/2)))
    truth = ((seq_Y-seq_Y.reshape(-1,1))==0).type(torch.float32)
    loss = -cos(att,truth).mean()
    loss.backward()
    optimizer.step()
    losses.append(loss.detach().cpu())

AssertionError: Torch not compiled with CUDA enabled